<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install gradio openpyxl --quiet

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import glob
import numpy as np
from io import BytesIO
import tempfile
import openpyxl

# Cargar CSV del PIB (suponiendo que está en /content)
csv_path = glob.glob("/content/API_*.csv")
assert csv_path, "❌ No se encontró archivo CSV válido"
csv_file = csv_path[0]

df = pd.read_csv(csv_file, skiprows=4)
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())
matriz_pib = df.set_index('Country Name').apply(pd.to_numeric, errors='coerce')

# Función principal para comparar PIB y exportar Excel
def comparar_y_exportar(pais1, pais2):
    try:
        # Filtrar los datos de los países seleccionados
        datos1 = matriz_pib.loc[pais1].dropna()
        datos2 = matriz_pib.loc[pais2].dropna()

        comparativa = pd.DataFrame({
            'Año': datos1.index.astype(int),
            pais1: datos1.values,
            pais2: datos2.reindex(datos1.index).values
        })

        # Reemplazar valores NaN e INF por 0 (más simple)
        comparativa = comparativa.replace([np.inf, -np.inf], 0).fillna(0)

        # Calcular el crecimiento de cada país
        comparativa[f"{pais1} Crec %"] = comparativa[pais1].pct_change() * 100
        comparativa[f"{pais2} Crec %"] = comparativa[pais2].pct_change() * 100
        comparativa = comparativa.round(2)

        # Gráfico de comparación PIB
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(comparativa['Año'], comparativa[pais1] / 1e9, label=pais1, marker='o')
        ax.plot(comparativa['Año'], comparativa[pais2] / 1e9, label=pais2, marker='o')
        ax.set_title(f'PIB: {pais1} vs {pais2}')
        ax.set_xlabel('Año')
        ax.set_ylabel('PIB (miles de millones USD)')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()

        # Guardar gráfico en memoria (BytesIO)
        imagen = BytesIO()
        fig.savefig(imagen, format='png')
        imagen.seek(0)

        # Crear archivo Excel temporal sin valores NaN o INF
        tmp_path = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx").name

        # Usar openpyxl para guardar el Excel
        with pd.ExcelWriter(tmp_path, engine='openpyxl') as writer:
            comparativa.to_excel(writer, index=False, sheet_name='Comparación PIB')

            workbook = writer.book
            worksheet = workbook['Comparación PIB']

            # Ajuste de columnas y estilo
            for col_num, value in enumerate(comparativa.columns.values):
                worksheet.cell(row=1, column=col_num + 1, value=value)
                worksheet.column_dimensions[openpyxl.utils.get_column_letter(col_num + 1)].width = 18

            # Formato de las celdas (números y porcentajes)
            for row_num, row in comparativa.iterrows():
                for col_num, value in enumerate(row):
                    if "Crec" in comparativa.columns[col_num]:
                        worksheet.cell(row=row_num + 2, column=col_num + 1, value=value)
                        worksheet.cell(row=row_num + 2, column=col_num + 1).number_format = '0.00%'
                    else:
                        worksheet.cell(row=row_num + 2, column=col_num + 1, value=value)

            # Insertar imagen del gráfico
            image_path = "/tmp/grafico.png"
            fig.savefig(image_path)
            worksheet.add_image(openpyxl.drawing.image.Image(image_path), 'H2')

        # IA: Sugerencias de países similares
        def sugerir(pais_ref):
            prom = matriz_pib.loc[pais_ref].mean()
            dist = matriz_pib.apply(lambda x: np.abs(x.mean() - prom), axis=1)
            return dist.sort_values().drop(index=pais_ref).head(3).index.tolist()

        sugeridos1 = sugerir(pais1)
        sugeridos2 = sugerir(pais2)

        texto = (
            f"🤖 IA Sugerencias:\n"
            f"- Países similares a **{pais1}**: {', '.join(sugeridos1)}\n"
            f"- Países similares a **{pais2}**: {', '.join(sugeridos2)}"
        )

        # Retornar gráficos y archivo Excel
        return fig, comparativa, texto, tmp_path
    except Exception as e:
        print(f"Error al procesar los datos: {e}")
        return None, None, f"Error: {e}", None

# Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🌎 Comparador de PIB con IA y Excel Estilizado")
    with gr.Row():
        dropdown1 = gr.Dropdown(choices=paises, label="País 1", value=paises[0])
        dropdown2 = gr.Dropdown(choices=paises, label="País 2", value=paises[1])
    boton = gr.Button("Comparar")
    grafico = gr.Plot()
    tabla = gr.Dataframe()
    texto = gr.Markdown()
    archivo = gr.File(label="📥 Descargar Excel")

    boton.click(fn=comparar_y_exportar,
                inputs=[dropdown1, dropdown2],
                outputs=[grafico, tabla, texto, archivo])

# Lanzar la app
demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a150494621be2aa89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
